# Performing Dimensionality Reduction with the built-in PCA Algorithm

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r unlabeled_normalized_df

In [ ]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()
bucket = session.default_bucket()

In [ ]:
from sagemaker import PCA

estimator = PCA(
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    num_components=2,
    sagemaker_session=session)

In [ ]:
data_np = unlabeled_normalized_df.values.astype('float32')
record_set = estimator.record_set(data_np)

In [ ]:
estimator.fit(record_set)

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

In [ ]:
results = predictor.predict(data_np)
results

In [ ]:
results[0].label['projection'].float32_tensor.values

In [ ]:
def extract_values(item):
    projection = item.label['projection']
    pair = projection.float32_tensor.values
    x = pair[0]
    y = pair[1]
    
    return {
        "x": x,
        "y": y
    }

In [ ]:
new_xs = []
new_ys = []

for result in results:
    x_and_y = extract_values(result)
    new_xs.append(x_and_y["x"])
    new_ys.append(x_and_y["y"])

In [ ]:
new_xs[0:5]

In [ ]:
new_ys[0:5]

In [ ]:
import pandas as pd

new_df = pd.DataFrame({
    "new_x": new_xs,
    "new_y": new_ys
})

new_df

In [ ]:
new_df.plot.scatter(x="new_x", y="new_y")